# Design Resistance of the Hot Rolled Steel Sections
# according to EN 1993-1-1:2005

2016-03 Pedro Biel

In [1]:
# Import Anaconda packages
# ------------------------

%pylab inline
import csv
import xlsxwriter

Populating the interactive namespace from numpy and matplotlib


Vendor:  Continuum Analytics, Inc.
Package: mkl
Message: trial mode expires in 9 days


In [2]:
# Precision for floating point output
# -----------------------------------

set_printoptions(precision = 2,   # No of decimals.
                 suppress = True) # Suppress printing of small floating point values using scientific notation.

---

## 1. Inputs

### EN 1993-1-1:2005

https://law.resource.org/pub/eu/eurocode/en.1993.1.1.2005.pdf

### Steel Sections

Steel sections according to ArcelorMittal.

http://sections.arcelormittal.com/products-services/products-ranges.html

    Serie                                             Section                    Dimensions       
    ----------------------------------------------------------------------------------------------------------------------
    European I beams                                  IPE                        EU 19-57
    European wide flange beams                        IPEAA, IPEA, IPEO, IPE 700 ArcelorMittal standard
    European wide flange beams                        HEA, HEB, HEM              EU 53-62
    European wide flange beams                        HE 1000                    ASTM A 6/A 6M
    European wide flange beams                        HEAA                       ArcelorMittal standard
    European extra wide flange beams                  HL                         ASTM A 6/A 6M; ArcelorMittal standard
    European extra wide flange beams                  HLZ                        ASTM A 6/A 6M; ArcelorMittal standard
    Wide flange columns                               HD 260/320                 ArcelorMittal standard
    Wide flange columns                               HD 360/400                 ASTM A 6/A 6M
    Wide flange bearing piles                         HP                         ArcelorMittal standard
    British universal beams                           UB                         BS 4-1:2005; ASTM A 6/A 6M; ArcelorMittal
    British universal columns                         UC                         BS 4-1:2005; ASTM A 6/A 6M; ArcelorMittal
    British universal bearing piles with wide flanges UBP                        BS 4-1:2005
    American wide flange beams                        W                          ASTM A 6/A 6M; ArcelorMittal standard
    American wide flange bearing piles                HP                         ASTM A 6/A 6M -11
    Russian hot rolled beams                          HG                         STO ASCHM 20-93; GOST 26020-83
    Japanese H sections                               H                          JIS G 3192: 2008; JIS A 5526: 2005
    European standard channels                        UPN                        DIN 1026-1: 2000, NF A 45-202: 1986
    Channels with parallel flanges                    UPE                        DIN 1026-2: 2002-10  
    American standard channels                        C                          ASTM A 6/A 6M

### csv File Reading

In [3]:
# ArcelorMittal csv file data.

sections_csv = []

with open('..\csv\sections_ArcelorMittal.csv', newline = '') as f_csv:
    reader = csv.reader(f_csv)
    for row in reader:
        if row[0].startswith('#'): continue # Skip the rows which begin with ‘#’.
        else: sections_csv.append(row)

### Arrays with the Sections Data

In [4]:
sections_arr = array(sections_csv) # Convert the csv list to an array.
sections = sections_arr.T          # Transpose the array.

In [5]:
section = array(sections[0])                 # Section range.
nominal = array(sections[1])                 # Nominal depth.
g =       array(sections[2], dtype = float)  # [kN/[mm] Weight per unit length.
h =       array(sections[3], dtype = float)  # [mm] Depth of section.
b =       array(sections[4], dtype = float)  # [mm] Width of section.
tw =      array(sections[5], dtype = float)  # [mm] Web thickness.
tf =      array(sections[6], dtype = float)  # [mm] Flange thickness.
r =       array(sections[7], dtype = float)  # [mm] Radius of root fillet.
A =       array(sections[8], dtype = float)  # [cm²] Area of section.
hi =      array(sections[9], dtype = float)  # [mm] Inner depth between flanges.
d =       array(sections[10], dtype = float) # [mm] Depth of straight portion of web.
O =       array(sections[11])                # [mm] Maximum bolt diameter.
pmin =    array(sections[12], dtype = float) # [mm] Minimum allowable edge distances.
pmax =    array(sections[13], dtype = float) # [mm] Maximum allowable edge distances.
AL =      array(sections[14], dtype = float) # [m²/m] Painting surface per unit length.
AG =      array(sections[15], dtype = float) # [m²/t] Painting surface per unit mass.
Iy =      array(sections[16], dtype = float) # [cm⁴] Second moment of area about the y axis.
Wely =    array(sections[17], dtype = float) # [cm³] Elastic section modulus about the y axis.
Wply =    array(sections[18], dtype = float) # [cm³] Plastic section modulus about the y axis.
iy =      array(sections[19], dtype = float) # [mm] Radius of gyration about the y axis.
AVz =     array(sections[20], dtype = float) # [cm²] Shear area in the z-direction.
Iz =      array(sections[21], dtype = float) # [cm⁴] Second moment of area about the z axis.
Welz =    array(sections[22], dtype = float) # [cm³] Elastic section modulus about the z axis.
Wplz =    array(sections[23], dtype = float) # [cm³] Plastic section modulus about the z axis.
iz =      array(sections[24], dtype = float) # [mm] Radius of gyration about the y axis.
ss =      array(sections[25], dtype = float) # [mm] Length of stiff bearing.
It =      array(sections[26], dtype = float) # [cm⁴] Torsion constant.
Iw =      array(sections[27], dtype = float) # [cm⁶] Warping constant.

In [6]:
print(unique(section))

['C' 'H' 'HD' 'HE' 'HEA' 'HEAA' 'HEB' 'HEC' 'HEM' 'HG' 'HL' 'HLA' 'HLAA'
 'HLB' 'HLM' 'HLR' 'HLZA' 'HLZB' 'HLZC' 'HLZD' 'HP' 'IPE' 'IPEA' 'IPEAA'
 'IPEO' 'IPEV' 'UB' 'UBP' 'UC' 'UPE' 'UPN' 'W']


### Yield Strength for Hot Rolled Structural Steel $f_y$ [MPa] (table 3.1)

In [7]:
fy_S235_EN = [235, 215] # [MPa] t <= 40 mm -> fy = 235 MPa; 40 mm < t <= 80 mm -> fy = 215 MPa.
fy_S275_EN = [275, 255] # [MPa] t <= 40 mm -> fy = 275 MPa; 40 mm < t <= 80 mm -> fy = 255 MPa.
fy_S355_EN = [355, 335] # [MPa] t <= 40 mm -> fy = 355 MPa; 40 mm < t <= 80 mm -> fy = 335 MPa.

In [8]:
fy_S235 = []
fy_S275 = []
fy_S355 = []

fy_S235 = [fy_S235_EN[0] if t <= 40 else fy_S235_EN[1] for t,t in zip(tw, tf)]
fy_S275 = [fy_S275_EN[0] if t <= 40 else fy_S275_EN[1] for t,t in zip(tw, tf)]
fy_S355 = [fy_S355_EN[0] if t <= 40 else fy_S355_EN[1] for t,t in zip(tw, tf)]

### Partial Factor for Resistance of Cross-Sections

In [9]:
γM0 = 1.00

---

## 2. Calculations

### $N_{pl,Rd}$ Design Plastic Resistance to Normal Forces $[kN]$

$$N_{pl,Rd} = \frac{A f_y}{\gamma_{M0}}$$

In [10]:
def NRd(As, fy, gM0): return As * fy / gM0 / 10

NplRd_S235 = NRd(A, fy_S235, γM0)
NplRd_S275 = NRd(A, fy_S275, γM0)
NplRd_S355 = NRd(A, fy_S355, γM0)

### $M_{el,Rd}$ Elastic Resistance to Bending Moments $[kNm]$

$$M_{el,Rd} = \frac{W_{el} f_y}{\gamma_{M0}}$$

In [11]:
def MRd(W, fy, gM0): return W * fy / gM0 / 1000

MelyRd_S235 = MRd(Wely, fy_S235, γM0)
MelyRd_S275 = MRd(Wely, fy_S275, γM0)
MelyRd_S355 = MRd(Wely, fy_S355, γM0)
                    
MelzRd_S235 = MRd(Welz, fy_S235, γM0)
MelzRd_S275 = MRd(Welz, fy_S275, γM0)
MelzRd_S355 = MRd(Welz, fy_S355, γM0)

### $M_{pl,Rd}$ Plastic Resistance to Bending Moments $[kNm]$

$$M_{pl,Rd} = \frac{W_{pl} f_y}{\gamma_{M0}}$$

In [12]:
MplyRd_S235 = MRd(Wply, fy_S235, γM0)
MplyRd_S275 = MRd(Wply, fy_S275, γM0)
MplyRd_S355 = MRd(Wply, fy_S355, γM0)
                    
MplzRd_S235 = MRd(Wplz, fy_S235, γM0)
MplzRd_S275 = MRd(Wplz, fy_S275, γM0)
MplzRd_S355 = MRd(Wplz, fy_S355, γM0)

### $V_{pl,Rd}$ Plastic Design Shear Resistance $[kN]$

$$V_{pl,Rd} = \frac{A_V f_y}{\sqrt{3} · \gamma_{M0}}$$

$A_{Vy}$ Is not included in the data file of ArcelorMittal. The value will be obtained from the formula: 

$$A_{Vy} = 2 b t_f$$

In [13]:
AVy = 2 * b * tf / 100 # [cm²] Shear area in the y-direction.

In [14]:
def VRd(Av, fy, gM0): return Av * fy / (sqrt(3) * gM0) / 10

VplyRd_S235 = VRd(AVy, fy_S235, γM0)
VplyRd_S275 = VRd(AVy, fy_S275, γM0)
VplyRd_S355 = VRd(AVy, fy_S355, γM0)

VplzRd_S235 = VRd(AVz, fy_S235, γM0)
VplzRd_S275 = VRd(AVz, fy_S275, γM0)
VplzRd_S355 = VRd(AVz, fy_S355, γM0)

---

## 3. Results

### Stack the Resistances of the Sections

In [15]:
stack_Rd = vstack((section, nominal,
                   NplRd_S235, NplRd_S275, NplRd_S275,
                   MelyRd_S235, MelyRd_S275, MelyRd_S355,
                   MelzRd_S235, MelzRd_S275, MelzRd_S355,
                   MplyRd_S235, MplyRd_S275, MplyRd_S355,
                   MplzRd_S235, MplzRd_S275, MplzRd_S355,
                   VplyRd_S235, VplyRd_S275, VplyRd_S355,
                   VplzRd_S235, VplzRd_S275, VplzRd_S355))

x = range(stack_Rd[0].size)
Rd = [[row[i] for row in stack_Rd] for i in x] # Transpose rows to columns.

### Save csv

In [16]:
with open('../csv/EN_DesResSteelSect (16-02).csv', 'w', newline='') as f_csv:
    writer = csv.writer(f_csv)
    writer.writerow(['# Design Resistance of the Steel Sections according to EN 1993-1-1:2005'])
    writer.writerow(['# ---------------------------------------------------------------------'])
    writer.writerow(['#'])
    writer.writerow(['# 2016-02-05 Pedro Biel'])
    writer.writerow(['#'])
    writer.writerow(['# Units in [kN]; [kNm]'])
    writer.writerow(['#'])
    writer.writerow(['# Section; Nominal; N.plRd_S235; N.plRd_S275; N.plRd_S275; M.elyRd_S235; M.elyRd_S275; M.elyRd_S355; M.elzRd_S235; M.elzRd_S275; M.elzRd_S355; M.plyRd_S235; M.plyRd_S275; M.plyRd_S355; M.plzRd_S235; M.plzRd_S275; M.plzRd_S355; V.plyRd_S235; V.plyRd_S275; V.plyRd_S355; V.plzRd_S235; V.plzRd_S275; V.plzRd_S355'])
    writer.writerow(['#'])
    writer.writerows(Rd)
    writer.writerow(['#'])
    writer.writerow(['# End of the document'])

### Save xlsx

In [17]:
# New book and new worksheet.
workbook = xlsxwriter.Workbook('../xlsx/EN_DesResSteelSect (16-03).xlsx', {'strings_to_numbers': True,})
workbook.set_properties({'title':   'Design Resistance of the Hot Rolled Steel Sections',
                         'subject': 'according to EN 1993-1-1:2005',
                         'author':  'Pedro Biel'})
worksheetS235 = workbook.add_worksheet('S235')
worksheetS275 = workbook.add_worksheet('S275')
worksheetS355 = workbook.add_worksheet('S355')

# Texts.
txt01 = 'Design Resistance of the Hot Rolled Steel Sections according to EN 1993-1-1:2005'
txt02 = '2016-03-02 Pedro Biel'
txt03 = 'Units in [kN], [kNm]'
txt04 = 'Section'
txt05 = 'Nominal'
txt06 = ('N.plRd', 'M.elyRd', 'M.elzRd', 'M.plyRd', 'M.plzRd', 'V.plyRd', 'V.plzRd')
txt07 = 'Steel grade: S235'
txt08 = 'Steel grade: S275'
txt09 = 'Steel grade: S355'

# Footer.
footerL = '&L&6&"Monospac821 BT" &F / http://nbviewer.jupyter.org/gist/PedroBiel'
footerR = '&R&6&"Monospac821 BT" &P/&N'

# Format.
format01 = workbook.add_format()    # Title.
format01.set_bold()
format01.set_font_size(14)

format02 = workbook.add_format()    # Section.
format02.set_bottom(1)

format03 = workbook.add_format()    # Nominal.
format03.set_align('center')
format03.set_bottom(1)

format04 = workbook.add_format()    # Other first rows.
format04.set_align('right')
format04.set_bottom(1)

format05 = workbook.add_format()    # Bottom border style.
format05.set_bottom(7)
format05.set_bottom_color('#A0A0A0')

format06 = workbook.add_format()    # Horizontal alignment right.
format06.set_align('right')
format06.set_bottom(7)
format06.set_bottom_color('#A0A0A0')

format07 = workbook.add_format()    # Format of the numbers of the lists.
format07.set_num_format('#,##0.00')
format07.set_bottom(7)
format07.set_bottom_color('#A0A0A0')

for worksheet in workbook.worksheets():                      # Return a list of the worksheet objects in the workbook.
    worksheet.set_column('A1:W1', 14)                        # Width of the column.
    worksheet.hide_gridlines(2)                              # Hide gridlines.
    worksheet.autofilter('A8:B8')                            # Filter.
    worksheet.freeze_panes('A9')                             # Create worksheet panes and mark them as frozen.
    worksheet.set_footer(footerL + footerR)                  # Footer.
    worksheet.protect('EN_CapResSecc', {'autofilter': True}) # Protect the worksheet.

    # First row.
    worksheet.write('A1', txt01, format01)
    worksheet.write('A2', txt02)
    worksheet.write('A6', txt03)
    worksheet.write('A8', txt04, format02)
    worksheet.write('B8', txt05, format03)
    worksheet.write_row('C8', txt06, format04)
    
    # Fill cells.
    worksheet.write_column('A9', section, format05)
    worksheet.write_column('B9', nominal, format06)
    
# Fill cells in worksheetS235.
worksheetS235.write('A4', txt07)
worksheetS235.write_column('C9', NplRd_S235,  format07)
worksheetS235.write_column('D9', MelyRd_S235, format07)
worksheetS235.write_column('E9', MelzRd_S235, format07)
worksheetS235.write_column('F9', MplyRd_S235, format07)
worksheetS235.write_column('G9', MplzRd_S235, format07)
worksheetS235.write_column('H9', VplyRd_S235, format07)
worksheetS235.write_column('I9', VplzRd_S235, format07) 

# Fill cells in worksheetS275.
worksheetS275.write('A4', txt08)
worksheetS275.write_column('C9', NplRd_S275,  format07)
worksheetS275.write_column('D9', MelyRd_S275, format07)
worksheetS275.write_column('E9', MelzRd_S275, format07)
worksheetS275.write_column('F9', MplyRd_S275, format07)
worksheetS275.write_column('G9', MplzRd_S275, format07)
worksheetS275.write_column('H9', VplyRd_S275, format07)
worksheetS275.write_column('I9', VplzRd_S275, format07)      
    
# Fill cells in worksheetS355.
worksheetS355.write('A4', txt09)
worksheetS355.write_column('C9', NplRd_S355,  format07)
worksheetS355.write_column('D9', MelyRd_S355, format07)
worksheetS355.write_column('E9', MelzRd_S355, format07)
worksheetS355.write_column('F9', MplyRd_S355, format07)
worksheetS355.write_column('G9', MplzRd_S355, format07)
worksheetS355.write_column('H9', VplyRd_S355, format07)
worksheetS355.write_column('I9', VplzRd_S355, format07)   

workbook.close()

## 4. Version

In [18]:
%load_ext version_information
%reload_ext version_information

%version_information csv, numpy, scipy, xlsxwriter

Software versions
Python 3.5.1 64bit [MSC v.1900 64 bit (AMD64)]
IPython 4.1.1
OS Windows 10 10.0.10586 SP0
csv 1.0
numpy 1.10.1
scipy 0.16.0
xlsxwriter 0.8.4
Sun Mar 06 18:34:53 2016 Hora estándar romance